In [ ]:
#default_exp tomography

In [ ]:
#export

from functools import cache
from typing import List, Tuple

from bokeh.palettes import palettes
from bokeh.plotting import figure, show, output_notebook
from nbdev.imports import *
from nbdev.showdoc import *
from scipy.linalg import expm

import numpy as np
# import plotly.graph_objects as go

output_notebook()

Loading BokehJS ...

# Tomography

> Implementation of quantum process tomography for system identification.

# Change of basis between computational basis and Pauli basis

$$
\newcommand{\expect}[1]{\langle#1\rangle}
\newcommand{\bra}[1]{\langle#1|}  % put argument into a "bra"
\newcommand{\ket}[1]{|#1\rangle}  % put argument into a "ket"
\newcommand{\inner}[2]{\bra{#1}#2\rangle}
\newcommand{\outer}[2]{\ket{#1}\bra{#2}}
\newcommand{\tr}{\text{Tr}}
\newcommand{\hc}[1]{#1^{\dagger}}
\newcommand{\timeevol}{\mathcal{U}}
\newcommand{\Uint}{U_{\text{int}}}
\newcommand{\Hint}{H_{\text{int}}}
$$

We will want to be able to go between the density matrix in the computational basis


$$
\rho_c = |a|^2 \outer{0}{0} + a b^* \outer{0}{1} + a^* b \outer{1}{0} + |b|^2 \outer{1}{1}
$$

and in the Pauli basis

$$
\begin{align*}
  \rho_p & = \frac{1}{2} \left[ \tr(\rho) I + \expect{X} X  + \expect{Y} Y + \expect{Z} Z \right] \\
  & = \frac{1}{2} \left[ (|a|^2 + |b|^2) I + (a b^* + a^* b) X  + (a b^* - a^* b) Y + (|a|^2 - |b|^2) Z \right]
\end{align*}
$$

By inspection of the above expression we can write down the matrices that take $\rho_c \rightleftarrows \rho_p$, as column vectors.

In [ ]:
#export

_comp_to_pauli_matrix = 0.5 * np.array(
    [
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 1j, -1j, 0],
        [1, 0, 0, -1],
    ]
)
_pauli_to_comp_matrix = np.array(
    [
        [1, 0, 0, 1],
        [0, 1, -1j, 0],
        [0, 1, 1j, 0],
        [1, 0, 0, -1]
    ]
)


def computational_to_pauli_state(density_mat: np.ndarray) -> np.ndarray:
    """Convert a density matrix in the computational basis """
    if np.shape(density_mat) == (4, ):
        return np.matmul(density_mat, _comp_to_pauli_matrix.T)
    elif np.shape(density_mat) == (4, 1):
        return np.matmul(_comp_to_pauli_matrix, density_mat)
    elif np.shape(density_mat) == (2, 2):
        return np.matmul(_comp_to_pauli_matrix, density_mat.reshape((4, 1))).reshape((2, 2))
    else:
        raise TypeError(f"density_mat has wrong shape {np.shape(density_mat)}")

def pauli_to_computational_state(density_mat: np.ndarray):
    if np.shape(density_mat) == (4, ):
        return np.matmul(density_mat, _pauli_to_comp_matrix.T)
    elif np.shape(density_mat) == (4, 1):
        return np.matmul(_pauli_to_comp_matrix, density_mat)
    elif np.shape(density_mat) == (2, 2):
        return np.matmul(_pauli_to_comp_matrix, density_mat.reshape((4, 1))).reshape((2, 2))
    else:
        raise TypeError(f"density_mat has wrong shape {np.shape(density_mat)}")

In [ ]:
# Round trip of comp to Pauli basis and back gives the identity
test_eq(_comp_to_pauli_matrix @ _pauli_to_comp_matrix, np.identity(4))

def test_comp_pauli_equality(comp, pauli):
    """Test equality of density matrices in computational and Pauli bases."""
    test_eq(computational_to_pauli_state(comp), pauli)
    test_eq(comp, pauli_to_computational_state(pauli))

# |0⟩ state density matrix is equal to 1/2 * (I + Z)
comp_density_matrix = [1, 0, 0, 0]
pauli_density_matrix = [0.5, 0, 0, 0.5]
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

# |1⟩ state density matrix is equal to 1/2 * (I - Z)
comp_density_matrix = [0, 0, 0, 1]
pauli_density_matrix = [0.5, 0, 0, -0.5]
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

# 1/2 * (|0⟩ + |1⟩) state density matrix is equal to 1/2 * (I + X)
comp_density_matrix = [0.5, 0.5, 0.5, 0.5]
pauli_density_matrix = [0.5, 0.5, 0, 0]
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

# 1/2 * (|0⟩ + i|1⟩) state density matrix is equal to 1/2 * (I + Y)
comp_density_matrix = [0.5, -0.5j, 0.5j, 0.5]
pauli_density_matrix = [0.5, 0, 0.5, 0]
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

# Works for column vectors
comp_density_matrix = np.array([[0.5], [-0.5j], [0.5j], [0.5]])
pauli_density_matrix = np.array([[0.5], [0], [0.5], [0]])
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

# Works for 2x2 matrices
comp_density_matrix = np.array([[0.5, -0.5j], [0.5j, 0.5]])
pauli_density_matrix = np.array([[0.5, 0], [0.5, 0]])
test_comp_pauli_equality(comp_density_matrix, pauli_density_matrix)

In [ ]:
#hide

class ComputationalBasis(L):
    """
    List (ordered set) of computational states of a collection of $N$ `Qudit`s.

    A computational state is represented internally as a tuple $(n_0, n_1, ..., n_{N-1})$, where
    each integer $n_i$ labels the state of one of the qudits. Each qudit can have its own
    dimensionality $d_i$, so we have $n_i \elem {0, ..., d_i-1}$.

    Note that when we describe a multi-qudit system, a choice always has to be made in the
    ordering of the indices of this tuple. To illustrate this and the choice made here, consider
    a quantum system consisting of a spin being used as a qubit ($d=2$), and a transmon being
    used as a qutrit ($d=3$).

    Since there are two elements in this system, we need two indices to describe it. Let $s=0, 1$
    index the computational states of the spin, and let $t=0, 1, 2$ index the computational state
    of the transmon. If we decide that $s$ will be the "first" index of the system, then we will
    have the following ways to represent an arbitrary computational state:



    To assign labels to our basis states we have to choose an ordering of the qudits, so let the
    spin be the first qudit and the transmons be the second. This means* that we will label a
    variable basis state as $|ts⟩$ and the basis itself will be ${\ket{00}, \ket{01}, \ket{02},
    \ket{10}, \ket{11}, \ket{12}}$.

    * See Robert Smith's (Someone shouts, "|01000>!" Who is Excited?)[https://arxiv.org/abs/1711.02086]
    """

    # def basis_bitstrings(self):
    #     return L(["".join([f"{x}" for x in reversed(range())]) for vec in self.computational_basis])

In [ ]:
#export

from notes.core.pauli import pauli_matrices as σ, pauli_names, PauliOperator


def pauli_transfer_matrix_unitary(unitary: np.ndarray) -> np.ndarray:
    """Compute the Pauli Transfer Matrix for a given unitary matrix `u`."""
    return np.real(
            np.array(
                [
                    [
                        np.trace(σ[j] @ unitary @ σ[i] @ unitary.conj().T)
                    for j in range(4)]
                for i in range(4)]
            )
        ) / len(unitary)


class QuantumGate:

    def __init__(self, computational_basis: List[Tuple], unitary_matrix: np.ndarray):
        self.computational_basis = computational_basis
        self.unitary_matrix = unitary_matrix
        self.dim = np.shape(unitary_matrix)[0]

    @property
    @cache
    def chi_matrix(self):
        """
        Compute the chi matrix representation of the gate.

        See Nielsen and Chuang, Quantum Computation and Quantum Information, 10th Anniversary Ed.,
        p. 391.
        """
        raise NotImplementedError

    @property
    @cache
    def pauli_transfer_matrix(self):
        return pauli_transfer_matrix_unitary(self.unitary_matrix)

In [ ]:
def rotate_su2(axis: Tuple[float, float, float], angle: float):
    """
    Calculate a rotation matrix in SU(2) given the axis and angle of the rotation.
    
    The rotation angle is in cycles.
    """
    return np.cos(np.pi*angle) * σ[0] + 1j* np.sin(np.pi*angle) * sum([axis[i-1] * σ[i] for i in range(1, 4)])

g = QuantumGate(
    computational_basis=[],
    unitary_matrix=expm(0.5j * np.pi/2 * σ[3]),
)

ptm = g.pauli_transfer_matrix
print(ptm)
print(ptm.shape[:2])

[[ 1.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]
(4, 4)


In [ ]:
import yaml

import colorcet as cc
from bokeh import palettes
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, GlyphRenderer, LinearColorMapper, Slider
from bokeh.plotting import figure, Figure, show
from bokeh.themes import Theme


def ptm_figure() -> Figure:
    """Prepare Bokeh rect plot to display a Pauli Transfer Matrix (PTM)."""
    fig = figure(
        x_range=pauli_names,
        y_range=list(reversed(pauli_names)),
        tooltips=[
            ("Channel", "@pauli_in → @pauli_out"),
            ("Trace", "@trace{1.3f}"),
        ],
        tools="hover,save",
        x_axis_location="above",
    )
    fig.grid.grid_line_color = None
    fig.grid.level = "overlay"
    fig.axis.axis_line_color = None
    fig.axis.major_tick_line_color = None
    fig.axis.major_label_text_font_size = "24px"
    fig.axis.major_label_standoff = 8
    fig.axis.axis_label_text_font_size = "24px"
    fig.xaxis.axis_label = "Input Pauli Component"
    fig.yaxis.axis_label = "Output Pauli Component"
#     fig.height = 550
#     fig.width = 550
    return fig


def ptm_data_source(ptm: np.ndarray) -> dict:
    """
    Build the `source` argument for the PTM plot.
    
    This is a dict with keys:
      `pauli_in`: the names of the input `PauliOperator`s
      `pauli_out`: the names of the output `PauliOperator`s
      `trace`: the values of each matrix element ($\frac{1}{d}\tr(\sigma_i \Lambda(\sigma(\sigma_j)))$)
    """
    return dict(
        pauli_in=[pauli_names[i] for j in range(4) for i in range(4)],
        pauli_out=[pauli_names[j] for j in range(4) for i in reversed(range(4))],
        trace=ptm.flatten(),
    )


def ptm_plot(ptm: np.ndarray) -> Tuple[Figure, GlyphRenderer]:
    """"""
    # TODO: Try a mapper that goes like (1/np.pi) * np.arcsin(z) to accentuate smaller variations
    mapper = LinearColorMapper(palette=list(reversed(cc.coolwarm)), low=-1, high=1)
    fig = ptm_figure()
    plot = fig.rect(
        x="pauli_in",
        y="pauli_out",
        source=ptm_data_source(ptm),
        fill_color={"field": "trace", "transform": mapper},
        line_color="lightgray",
        width=1,
        height=1,
    )
    fig.add_layout(
        ColorBar(
            color_mapper=mapper,
            location=(0,0),
        ),
        'right')
    return fig, plot


def update_ptm_plot(attr, old, new):
    """"""
    x = unit_vector_x.value
    y = unit_vector_y.value
    z = np.sqrt(1 - x**2 - y**2)
    rotation_matrix = rotate_su2((x, y, z), angle.value)
    ptm = pauli_transfer_matrix_unitary(rotation_matrix)
    source.data = ptm_data_source(ptm)

# u = rotate_su2((1, 0, 0), np.pi/2)
# ptm = pauli_transfer_matrix_unitary(u)
# fig, plot = ptm_plot(ptm)
# show(fig)

# unit_vector_x = Slider(title="Rotation axis x coord", value=0.0, start=-1.0, end=1.0, step=0.01)
# unit_vector_y = Slider(title="Rotation axis y coord", value=0.0, start=-1.0, end=1.0, step=0.01)
# angle = Slider(title="Rotation angle", value=0.0, start=0.0, end=2*np.pi, step=0.01)
# for slider in [unit_vector_x, unit_vector_y, angle]:
#     slider.on_change('value', update_ptm_plot)

# inputs = column(unit_vector_x, unit_vector_y, angle)
# show(row(inputs, fig, width=800))

In [ ]:
def bkapp(doc):
    """
    Magically put a Bokeh server into thy notebook.
    
    Credit: https://github.com/bokeh/bokeh/blob/2.2.3/examples/howto/server_embed/notebook_embed.ipynb.
    """
    u = rotate_su2((0, 0, 1), 0)
    ptm = pauli_transfer_matrix_unitary(u)
    fig, plot = ptm_plot(ptm)
    
    def update_ptm_plot(attr, old, new):
        x = unit_vector_x.value
        y = unit_vector_y.value
        z = np.sqrt(1 - x**2 - y**2)
        rotation_matrix = rotate_su2((x, y, z), angle.value)
        ptm = pauli_transfer_matrix_unitary(rotation_matrix)
        plot.data_source.data = ptm_data_source(ptm)

    unit_vector_x = Slider(title="Rotation axis x coord", value=0.0, start=-1.0, end=1.0, step=0.01)
    unit_vector_y = Slider(title="Rotation axis y coord", value=0.0, start=-1.0, end=1.0, step=0.01)
    angle = Slider(title="Rotation angle [cycles]", value=0.0, start=0.0, end=2.0, step=0.01)
    for slider in [unit_vector_x, unit_vector_y, angle]:
        slider.on_change('value', update_ptm_plot)
    inputs = column(unit_vector_x, unit_vector_y, angle)
    doc.add_root(column(inputs, fig))
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [ ]:
show(bkapp, notebook_url="http://localhost:8890")

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted core.pauli.ipynb.
Converted index.ipynb.
Converted tomography.ipynb.
